In [5]:
%load_ext autoreload

import dataset
import numpy as np
import pydotplus
from IPython.display import Image
from sklearn.externals.six import StringIO
from sklearn.tree import DecisionTreeRegressor, export_graphviz

import hyperhyper as hy

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
for y in [2010, 2012, 2014, 2016, 2018]:
    b = hy.Bunch(f'/mnt/data2/ptf/bunches/bi_{y}_decay_4')
    best = b.results()[0]
    print(best['pair_args__decay_rate'])
    print(best['eig'])
    print(best['neg'])
    print()

0.33
0.35000000000000003
1.2999999999999998

0.33
0.25
1.4999999999999998

0.36000000000000004
0.35000000000000003
0.7

0.36000000000000004
0.35000000000000003
0.7999999999999999

0.33
0.15000000000000002
0.7



/home/filter/anaconda3/envs/hyperhyper/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/home/filter/anaconda3/envs/hyperhyper/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/home/filter/anaconda3/envs/hyperhyper/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Tec

In [19]:
b.results()

[OrderedDict([('id', 330),
              ('method', 'svd'),
              ('impl', 'scipy'),
              ('pair_args__window', 10.0),
              ('pair_args__dynamic_window', 'decay'),
              ('pair_args__decay_rate', 0.33),
              ('pair_args__delete_oov', 1.0),
              ('pair_args__subsample', 'deter'),
              ('pair_args__subsample_factor', 7e-05),
              ('neg', 1.4999999999999998),
              ('eig', 0.25),
              ('dim', 500.0),
              ('micro_results', 0.5433854673263544),
              ('macro_results', 0.6038202406445505),
              ('ws353rel_score', 0.6515144519028375),
              ('ws353rel_oov', 0.09126984126984126),
              ('ws353rel_fullscore', 0.5920508312926579),
              ('schm280_score', 0.7154801498472478),
              ('schm280_oov', 0.09642857142857143),
              ('schm280_fullscore', 0.6464874211119775),
              ('gur65_score', 0.6088366904135839),
              ('gur65_oov', 

In [20]:
db = dataset.connect("sqlite:////mnt/data2/ptf/bunches/bi_2010_decay_4/results.db")

In [130]:
# db = dataset.connect("sqlite:////mnt/data2/ptf/bunches/bi_2018/results.db")

In [131]:
def get_samples(db):
    statement = "SELECT distinct eig, dim, neg, pair_args__decay_rate, pair_args__window, micro_results, pair_args__subsample_factor, pair_args__delete_oov FROM experiments ORDER BY micro_results DESC LIMIT 300"
    for row in db.query(statement):
        # replace None values with defaults (TODO: should be fixed in module)
        e = row["eig"]
        if e is None:
            e = 0

        d = row["dim"]
        if d is None:
            d = 500
        try:
            window = int.from_bytes(row["pair_args__window"], 'little')
        except:
            window = row["pair_args__window"]
        print(window, row)

        yield [window, row["pair_args__decay_rate"],row["pair_args__subsample_factor"],row["pair_args__delete_oov"], row["neg"], e, d], row["micro_results"]

In [132]:
X = []
Y = []
sap = list(get_samples(db))
for x, y in sap:
    X.append(x)
    Y.append(y)

10.0 OrderedDict([('eig', 0.35000000000000003), ('dim', 500.0), ('neg', 1.2999999999999998), ('pair_args__decay_rate', 0.33), ('pair_args__window', 10.0), ('micro_results', 0.5486674539152988), ('pair_args__subsample_factor', 7e-05), ('pair_args__delete_oov', 1.0)])
10.0 OrderedDict([('eig', 0.30000000000000004), ('dim', 500.0), ('neg', 1.4), ('pair_args__decay_rate', 0.36000000000000004), ('pair_args__window', 10.0), ('micro_results', 0.5472864569677228), ('pair_args__subsample_factor', 7e-05), ('pair_args__delete_oov', 1.0)])
10.0 OrderedDict([('eig', 0.35000000000000003), ('dim', 500.0), ('neg', 1.2999999999999998), ('pair_args__decay_rate', 0.36000000000000004), ('pair_args__window', 10.0), ('micro_results', 0.5472452633275545), ('pair_args__subsample_factor', 7e-05), ('pair_args__delete_oov', 1.0)])
10.0 OrderedDict([('eig', 0.35000000000000003), ('dim', 500.0), ('neg', 1.2999999999999998), ('pair_args__decay_rate', 0.34), ('pair_args__window', 10.0), ('micro_results', 0.547195201

In [51]:
zip(X,d Y)

SyntaxError: invalid syntax (<ipython-input-51-0210217c8012>, line 1)

In [52]:
tree = DecisionTreeRegressor(min_samples_leaf=1)

In [7]:
tree.fit(X, Y)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

In [8]:
tree.score(X, Y)

0.99999152642929

In [9]:
dot_data = StringIO()
export_graphviz(tree, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True, feature_names=['window', 'neg', 'eig', 'd'])
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

ValueError: Length of feature_names, 4 does not match number of features, 6